# Create a database of organic molecules based on TCI scraped data

In [94]:
## Imports

In [1]:
import pandas as pd
import numpy as np
from rdkit.Chem import AllChem as Chem
from tqdm.notebook import tqdm
from utils import *
from config import host, port, database, user, password

# Load the data

In [2]:
%cd data
!ls 

/home/stephy/Learning_DataScience/Chemical_db/TCI_database_app/data
'TCI_available stock.json'		 TCI_properties.json
'TCI_compound identifications.json'	'TCI_Related Laws.json'
'TCI_general information.json'		 TCI_specifications.json
'TCI_GHS precautionary statement.json'	'TCI_Transport information.json'
 TCI_Other.json


## Compound Identifications Data

In [3]:
df = pd.read_json('TCI_compound identifications.json', orient ='records', compression = 'infer')

### Convert smiles with RDKit and attach to the data

In [4]:
df['rdkit_smiles'] = df.SMILES_by_PubChem.apply(get_cansmi) # Chem.MolToSmiles(Chem.MolFromSmiles(smi), canonical=True)

In [6]:
df[['SMILES_by_PubChem', 'rdkit_smiles']].sample(10)

,SMILES_by_PubChem,rdkit_smiles
1487,CC1=C(C=NC=C1)N,Cc1ccncc1N
1212,CCCCCC1=CC=NC=C1,CCCCCc1ccncc1
930,CC(=O)NC1CCNC1,CC(=O)NC1CCNC1
962,CC(=O)OC1=CN=CC=C1,CC(=O)Oc1cccnc1
1168,COC1=CC=CC(=C1)C(=O)NN,COc1cccc(C(=O)NN)c1
1440,CC(=O)NC1=CC(=C(C=C1)N)C(F)(F)F,CC(=O)Nc1ccc(N)c(C(F)(F)F)c1
375,CCCCCC(C(=O)C)C(=O)OCC,CCCCCC(C(C)=O)C(=O)OCC
1290,C1=CC(=NC=C1Br)N,Nc1ccc(Br)cn1
708,C1=CC(=C(C=C1N)C(=O)O)[N+](=O)[O-],Nc1ccc([N+](=O)[O-])c(C(=O)O)c1
1225,C1=CC(=C(C(=C1)Cl)N)C(F)(F)F,Nc1c(Cl)cccc1C(F)(F)F


In [7]:
df.columns = ['name', 'cas', 'reaxys_registry_number', 'pubchem_id',
       'sdbs_aist_spectral_db', 'merck_index', 'mdl_number',
       'pubchem_smiles', 'id', 'grade', 'cas_rn', 'colour_index',
       'enzyme_commission_number', 'rdkit_smiles']

In [8]:
df.sample(5)

,name,cas,reaxys_registry_number,pubchem_id,sdbs_aist_spectral_db,merck_index,mdl_number,pubchem_smiles,id,grade,cas_rn,colour_index,enzyme_commission_number,rdkit_smiles
1,Ethyl Abietate,631-71-0,NaN,87561708.0,NaN,NaN,mfcd00028860,CCOC(=O)C1(CCCC2(C1CC=C3C2CCC(=C3)C(C)C)C)C,A0002,None,None,NaN,None,CCOC(=O)C1(C)CCCC2(C)C3CCC(C(C)C)=CC3=CCC12
1106,(R)-(-)-2-Aminobutyric Acid,2623-91-8,1720934.0,87562832.0,NaN,NaN,mfcd00064414,CCC(C(=O)O)N,A1377,None,None,NaN,None,CCC(N)C(=O)O
24,tert-Butyl Acetate,540-88-5,1699506.0,87561729.0,583.0,1537.0,mfcd00008807,CC(=O)OC(C)(C)C,A0026,None,None,NaN,None,CC(=O)OC(C)(C)C
778,3-Amino-5-methylisoxazole,1072-67-9,108110.0,87562478.0,23830.0,NaN,mfcd00003155,CC1=CC(=NO1)N,A0980,None,None,NaN,None,Cc1cc(N)no1
265,"Disodium Anthraquinone-2,6-disulfonate",853-68-9,3582949.0,87561961.0,7271.0,NaN,mfcd00001230,C1=CC2=C(C=C1S(=O)(=O)[O-])C(=O)C3=C(C2=O)C=C(...,A0308,None,None,NaN,None,O=C1c2ccc(S(=O)(=O)[O-])cc2C(=O)c2ccc(S(=O)(=O...


### Columns of the SQL table

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   name                      2000 non-null   object 
 1   cas                       2000 non-null   object 
 2   reaxys_registry_number    1933 non-null   float64
 3   pubchem_id                1943 non-null   float64
 4   sdbs_aist_spectral_db     918 non-null    float64
 5   merck_index               357 non-null    float64
 6   mdl_number                1874 non-null   object 
 7   pubchem_smiles            1972 non-null   object 
 8   id                        2000 non-null   object 
 9   grade                     41 non-null     object 
 10  cas_rn                    51 non-null     object 
 11  colour_index              45 non-null     float64
 12  enzyme_commission_number  9 non-null      object 
 13  rdkit_smiles              1972 non-null   object 
dtypes: float

In [17]:
columns = \
[('name', 'TEXT', '', 'Name of the compound'),
 ('cas', 'TEXT', '', 'CAS identification'),
 ('reaxys_registry_number', 'REAL', '', 'Identifier for chemical substance in Reaxys (Beilstein Registry Number)'),
 ('pubchem_id', 'REAL', '', 'Identifier from database of chemical molecules and their activities in biological assays PubChem ID'),
 ('sdbs_aist_spectral_db', 'REAL', '', 'Spectral Database for Organic Compounds'),
 ('merck_index', 'REAL', '', ' Merck Index for authoritative information on chemicals, drugs and biologicals'), 
 ('mdl_number', 'TEXT', '', 'MLD unique identification number for each reaction and variation.'),
 ('pubchem_smiles', 'TEXT', '', 'SMILES identifier from Pubchem'),
 ('id', 'TEXT', 'PRIMARY KEY', 'TCI unique id'),
 ('grade', 'TEXT', '', 'Grade refering to the purity of the chemical'),
 ('cas_rn', 'TEXT', '', 'CAS RN unique identifier that provides an unambiguous means to distinguish chemical substances or molecular structures'), 
 ('colour_index', 'REAL', '', 'Colour Index Generic Name describes a commercial product by its recognised usage class, its hue and a serial number'),
 ('enzyme_commission_number', 'TEXT', '', 'Enzyme Commission numerical classification scheme for enzymes, based on the chemical reactions they catalyze'),
 ('rdkit_smiles', 'TEXT', '', 'SMILES identifier from RDKit')
]

### Details

| column name | data type | table constraint | description |
| ---- | ---- | ---- | ---- |
|name | TEXT | - | Name of the compound |
|CAS|TEXT|-|CAS identification|
|reaxys registry number|REAL|-|Identifier for chemical substance in Reaxys (Beilstein Registry Number)|
|pubchem substance id|REAL|-|Identifier from database of chemical molecules and their activities in biological assays PubChem ID|
|sdbs (aist spectral db)|REAL|-|Spectral Database for Organic Compounds|
|merck index (14)|REAL|-| Merck Index for authoritative information on chemicals, drugs and biologicals| 
|mdl number|TEXT|-|MLD unique identification number for each reaction and variation.|
|SMILES_by_PubChem|TEXT|-|SMILES identifier from Pubchem|
|id|TEXT|PRIMARY KEY|TCI unique id|
|grade|TEXT|-|Grade refering to the purity of the chemical|
|related cas rn|TEXT|-|CAS RN unique identifier that provides an unambiguous means to distinguish chemical substances or molecular structures| 
|colour index|REAL|-|Colour Index Generic Name describes a commercial product by its recognised usage class, its hue and a serial number|
|enzyme commission number|TEXT|-|Enzyme Commission numerical classification scheme for enzymes, based on the chemical reactions they catalyze|
|rdkit_smiles | TEXT | - | SMILES identifier from RDKit |

### Create table and upload to server

In [11]:
from sqlalchemy import create_engine
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')
df.to_sql('compound_identification', engine,  index=False)

In [12]:
# define a primary Key
with engine.connect() as con:
    con.execute('ALTER TABLE compound_identification ADD PRIMARY KEY ("id");')

In [18]:
#create an index
with engine.connect() as con:
    con.execute(f"CREATE INDEX compound_identification_index ON compound_identification({','.join([col[0] for col in columns])});")

### Test the connection

In [27]:
%%time
table_retrieved = engine.execute("SELECT * FROM compound_identification").fetchall()

CPU times: user 29.2 ms, sys: 3.39 ms, total: 32.6 ms
Wall time: 26 s


In [25]:
%%time
df_retrieved = pd.read_sql("SELECT * FROM compound_identification", engine)

CPU times: user 33.3 ms, sys: 11.8 ms, total: 45.1 ms
Wall time: 19.6 s


In [26]:
df_retrieved.sample(10)

,name,cas,reaxys_registry_number,pubchem_id,sdbs_aist_spectral_db,merck_index,mdl_number,pubchem_smiles,id,grade,cas_rn,colour_index,enzyme_commission_number,rdkit_smiles
1794,Amino-5-chloro-3-methylbenzoic Acid,20776-67-4,3530413.0,160871357.0,NaN,NaN,mfcd02358895,CC1=CC(=CC(=C1N)C(=O)O)Cl,A2399,None,None,NaN,None,Cc1cc(Cl)cc(C(=O)O)c1N
296,"Amino-1,5-naphthalenedisulfonic Acid Disodium ...",14170-43-5,4117152.0,87561990.0,NaN,NaN,mfcd00070487,C1=CC2=C(C=C(C=C2S(=O)(=O)[O-])N)C(=C1)S(=O)(=...,A0339,None,None,NaN,None,Nc1cc(S(=O)(=O)[O-])c2cccc(S(=O)(=O)[O-])c2c1....
1559,"2-Amino-4,5-dimethylthiazole Hydrobromide",7170-76-5,3690705.0,87559841.0,52113.0,NaN,mfcd00035212,CC1=C(SC(=N1)N)C.Br,A2062,None,None,NaN,None,Br.Cc1nc(N)sc1C
1096,"2-Amino-4,4,4-trifluorobutyric Acid Hydrochloride",15959-93-0,19545620.0,87562822.0,NaN,NaN,mfcd00671488,C(C(C(=O)O)N)C(F)(F)F,A1367,None,None,NaN,None,NC(CC(F)(F)F)C(=O)O
827,2-Amino-9-fluorenone,3096-57-9,2804184.0,87562532.0,12164.0,NaN,mfcd00001157,C1=CC=C2C(=C1)C3=C(C2=O)C=C(C=C3)N,A1040,None,None,NaN,None,Nc1ccc2c(c1)C(=O)c1ccccc1-2
1502,Amino-2-chlorophenol,6358-06-1,2081092.0,87559209.0,51618.0,NaN,mfcd01707858,C1=CC(=C(C=C1N)O)Cl,A1976,None,None,NaN,None,Nc1ccc(Cl)c(O)c1
1908,4-Amino-2-methoxypyridine,20265-39-8,115200.0,172089020.0,NaN,NaN,mfcd06738657,COC1=NC=CC(=C1)N,A2543,None,None,NaN,None,COc1cc(N)ccn1
1897,Amiodarone Hydrochloride,19774-82-4,4776323.0,172088928.0,NaN,482.0,mfcd00069204,CCCCC1=C(C2=CC=CC=C2O1)C(=O)C3=CC(=C(C(=C3)I)O...,A2530,None,1951-25-3,NaN,None,CCCCc1oc2ccccc2c1C(=O)c1cc(I)c(OCCN(CC)CC)c(I)...
502,2-Aminothiazole,96-50-4,105738.0,87562204.0,3370.0,479.0,mfcd00005325,C1=CSC(=N1)N,A0633,None,None,NaN,None,Nc1nccs1
1153,Sodium 4-Acetylbenzenesulfonate,61827-67-6,4218686.0,87562884.0,22400.0,NaN,mfcd00007510,CC(=O)C1=CC=C(C=C1)S(=O)(=O)[O-].[Na+],A1430,None,None,NaN,None,CC(=O)c1ccc(S(=O)(=O)[O-])cc1.[Na+]


### Test different queries

In [31]:
test_smiles = engine.execute("SELECT * FROM compound_identification WHERE rdkit_smiles>'[n,o,s]1cccc1' ").fetchall()

In [32]:
df_test_smiles = pd.DataFrame(test_smiles)

In [95]:
df_test_smiles.sample(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
121,9-Anthraceneboronic Acid,100622-34-2,3301031.0,135727333.0,NaN,NaN,mfcd03425925,B(C1=C2C=CC=CC2=CC3=CC=CC=C13)(O)O,A2328,None,None,NaN,None,OB(O)c1c2ccccc2cc2ccccc12
60,Dibenzyl Adipate,2451-84-5,1892058.0,87562191.0,7556.0,NaN,mfcd00059667,C1=CC=C(C=C1)COC(=O)CCCCC(=O)OCC2=CC=CC=C2,A0618,None,None,NaN,None,O=C(CCCCC(=O)OCc1ccccc1)OCc1ccccc1
91,4-(1-Adamantyl)phenol,29799-07-3,2054509.0,87562847.0,30018.0,NaN,mfcd00168143,C1C2CC3CC1CC(C2)(C3)C4=CC=C(C=C4)O,A1392,None,None,NaN,None,Oc1ccc(C23CC4CC(CC(C4)C2)C3)cc1
119,Ammonium Thioglycolate,5421-46-5,11213443.0,87561582.0,NaN,NaN,mfcd00137451,C(C(=O)[O-])S.[NH4+],A2217,None,None,NaN,None,O=C([O-])CS.[NH4+]
110,",3-Anthracenedicarboxylic Anhydride",6812-14-2,206364.0,87558132.0,NaN,NaN,None,C1=CC=C2C=C3C=C4C(=CC3=CC2=C1)C(=O)OC4=O,A1834,None,None,NaN,None,O=C1OC(=O)c2cc3cc4ccccc4cc3cc21


## Available Stock

In [34]:
df_2 = pd.read_json('TCI_available stock.json', orient ='records', compression = 'infer')

In [57]:
df_2.columns = ['id','Stock25G_price','Stock25G_Saitama_Kawaguchi','Stock25G_Hyogo_Amagasaki',
 'Stock25G_Stock_in_other_WH','Stock500G_price','Stock500G_Saitama_Kawaguchi',
 'Stock500G_Hyogo_Amagasaki','Stock500G_Stock_in_other_WH','Stock100G_price',
 'Stock100G_Saitama_Kawaguchi','Stock100G_Hyogo_Amagasaki',
 'Stock100G_Stock_in_other_WH','Stock1G_price','Stock1G_Saitama_Kawaguchi',
 'Stock1G_Hyogo_Amagasaki','Stock1G_Stock_in_other_WH','Stock5G_price',
 'Stock5G_Saitama_Kawaguchi','Stock5G_Hyogo_Amagasaki',
 'Stock5G_Stock_in_other_WH','Stock250G_price','Stock250G_Saitama_Kawaguchi',
 'Stock250G_Hyogo_Amagasaki','Stock250G_Stock_in_other_WH','Stock10G_price',
 'Stock10G_Saitama_Kawaguchi','Stock10G_Hyogo_Amagasaki',
 'Stock10G_Stock_in_other_WH','Stockprepared_after_order_item','Stock200G_price',
 'Stock200G_Saitama_Kawaguchi','Stock200G_Hyogo_Amagasaki',
 'Stock200G_Stock_in_other_WH','Stock300G_price','Stock300G_Saitama_Kawaguchi',
 'Stock300G_Hyogo_Amagasaki','Stock300G_Stock_in_other_WH','Stock20G_price',
 'Stock20G_Saitama_Kawaguchi','Stock20G_Hyogo_Amagasaki',
 'Stock20G_Stock_in_other_WH','Stock50G_price','Stock50G_Saitama_Kawaguchi',
 'Stock50G_Hyogo_Amagasaki','Stock50G_Stock_in_other_WH']

In [58]:
df_2.sample(5)

,id,Stock25G_price,Stock25G_Saitama_Kawaguchi,Stock25G_Hyogo_Amagasaki,Stock25G_Stock_in_other_WH,Stock500G_price,Stock500G_Saitama_Kawaguchi,Stock500G_Hyogo_Amagasaki,Stock500G_Stock_in_other_WH,Stock100G_price,...,Stock300G_Hyogo_Amagasaki,Stock300G_Stock_in_other_WH,Stock20G_price,Stock20G_Saitama_Kawaguchi,Stock20G_Hyogo_Amagasaki,Stock20G_Stock_in_other_WH,Stock50G_price,Stock50G_Saitama_Kawaguchi,Stock50G_Hyogo_Amagasaki,Stock50G_Stock_in_other_WH
1501,A1975,None,None,None,None,None,None,None,None,None,...,None,None,None,NaN,None,None,None,NaN,None,None
1966,A2647,None,None,None,None,None,None,None,None,None,...,None,None,None,NaN,None,None,None,NaN,None,None
1036,A1301,None,None,None,None,None,None,None,None,None,...,None,None,None,NaN,None,None,None,NaN,None,None
915,A1153,None,None,None,None,None,None,None,None,"3,100",...,None,None,None,NaN,None,None,None,NaN,None,None
1670,A2220,None,None,None,None,None,None,None,None,None,...,None,None,None,NaN,None,None,None,NaN,None,None


In [59]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 46 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              2000 non-null   object 
 1   Stock25G_price                  1236 non-null   object 
 2   Stock25G_Saitama_Kawaguchi      1236 non-null   object 
 3   Stock25G_Hyogo_Amagasaki        1236 non-null   object 
 4   Stock25G_Stock_in_other_WH      1236 non-null   object 
 5   Stock500G_price                 369 non-null    object 
 6   Stock500G_Saitama_Kawaguchi     369 non-null    object 
 7   Stock500G_Hyogo_Amagasaki       369 non-null    object 
 8   Stock500G_Stock_in_other_WH     369 non-null    object 
 9   Stock100G_price                 217 non-null    object 
 10  Stock100G_Saitama_Kawaguchi     217 non-null    object 
 11  Stock100G_Hyogo_Amagasaki       217 non-null    object 
 12  Stock100G_Stock_in_other_WH     21

In [60]:
columns_2 = \
[('id', 'TEXT', 'PRIMARY KEY', 'TCI unique id'),
 ('Stock25G_price', 'TEXT', '', 'Price for the specified quatity of product'),
 ('Stock25G_Saitama_Kawaguchi','TEXT', '', 'Number of goods available in Saitama_Kawaguchi'),
 ('Stock25G_Hyogo_Amagasaki','TEXT', '', 'Number of goods available in Hyogo_Amagasaki'),
 ('Stock25G_Stock_in_other_WH', 'TEXT', ' Number of goods in other Warehouses'),
 ('Stock500G_price', 'TEXT', '', 'Price for the specified quatity of product'),
 ('Stock500G_Saitama_Kawaguchi','TEXT', '', 'Number of goods available in Saitama_Kawaguchi'),
 ('Stock500G_Hyogo_Amagasaki','TEXT', '', 'Number of goods available in Hyogo_Amagasaki'),
 ('Stock500G_Stock_in_other_WH', 'TEXT', ' Number of goods in other Warehouses'),
 ('Stock100G_price', 'TEXT', '', 'Price for the specified quatity of product'),
 ('Stock100G_Saitama_Kawaguchi','TEXT', '', 'Number of goods available in Saitama_Kawaguchi'),
 ('Stock100G_Hyogo_Amagasaki','TEXT', '', 'Number of goods available in Hyogo_Amagasaki'),
 ('Stock100G_Stock_in_other_WH', 'TEXT', ' Number of goods in other Warehouses'),
 ('Stock1G_price', 'TEXT', '', 'Price for the specified quatity of product'),
 ('Stock1G_Saitama_Kawaguchi','TEXT', '', 'Number of goods available in Saitama_Kawaguchi'),
 ('Stock1G_Hyogo_Amagasaki','TEXT', '', 'Number of goods available in Hyogo_Amagasaki'),
 ('Stock1G_Stock_in_other_WH', 'TEXT', ' Number of goods in other Warehouses'),
 ('Stock5G_price', 'TEXT', '', 'Price for the specified quatity of product'),
 ('Stock5G_Saitama_Kawaguchi','TEXT', '', 'Number of goods available in Saitama_Kawaguchi'),
 ('Stock5G_Hyogo_Amagasaki','TEXT', '', 'Number of goods available in Hyogo_Amagasaki'),
 ('Stock5G_Stock_in_other_WH', 'TEXT', ' Number of goods in other Warehouses'),
 ('Stock250G_price', 'TEXT', '', 'Price for the specified quatity of product'),
 ('Stock250G_Saitama_Kawaguchi','TEXT', '', 'Number of goods available in Saitama_Kawaguchi'),
 ('Stock250G_Hyogo_Amagasaki','TEXT', '', 'Number of goods available in Hyogo_Amagasaki'),
 ('Stock250G_Stock_in_other_WH', 'TEXT', ' Number of goods in other Warehouses'),
 ('Stock10G_price', 'TEXT', '', 'Price for the specified quatity of product'),
 ('Stock10G_Saitama_Kawaguchi','TEXT', '', 'Number of goods available in Saitama_Kawaguchi'),
 ('Stock10G_Hyogo_Amagasaki','TEXT', '', 'Number of goods available in Hyogo_Amagasaki'),
 ('Stock10G_Stock_in_other_WH', 'TEXT', ' Number of goods in other Warehouses'),
 ('Stockprepared_after_order_item', 'TEXT', '',  'Prepared after item'),
 ('Stock200G_price', 'TEXT', '', 'Price for the specified quatity of product'),
 ('Stock200G_Saitama_Kawaguchi','TEXT', '', 'Number of goods available in Saitama_Kawaguchi'),
 ('Stock200G_Hyogo_Amagasaki','TEXT', '', 'Number of goods available in Hyogo_Amagasaki'),
 ('Stock200G_Stock_in_other_WH', 'TEXT', ' Number of goods in other Warehouses'),
 ('Stock300G_price', 'TEXT', '', 'Price for the specified quatity of product'),
 ('Stock300G_Saitama_Kawaguchi','TEXT', '', 'Number of goods available in Saitama_Kawaguchi'),
 ('Stock300G_Hyogo_Amagasaki','TEXT', '', 'Number of goods available in Hyogo_Amagasaki'),
 ('Stock300G_Stock_in_other_WH', 'TEXT', ' Number of goods in other Warehouses'),
 ('Stock20G_price', 'TEXT', '', 'Price for the specified quatity of product'),
 ('Stock20G_Saitama_Kawaguchi','TEXT', '', 'Number of goods available in Saitama_Kawaguchi'),
 ('Stock20G_Hyogo_Amagasaki','TEXT', '', 'Number of goods available in Hyogo_Amagasaki'),
 ('Stock20G_Stock_in_other_WH', 'TEXT', ' Number of goods in other Warehouses'),
 ('Stock50G_price', 'TEXT', '', 'Price for the specified quatity of product'),
 ('Stock50G_Saitama_Kawaguchi','TEXT', '', 'Number of goods available in Saitama_Kawaguchi'),
 ('Stock50G_Hyogo_Amagasaki','TEXT', '', 'Number of goods available in Hyogo_Amagasaki'),
 ('Stock50G_Stock_in_other_WH', 'TEXT', ' Number of goods in other Warehouses')
]

### Details

| column name | data type | table constraint | description |
| ---- | ---- | ---- | ---- |
|id|TEXT|PRIMARY KEY|TCI unique id|
|Stock25G_price|TEXT|-|Price for the specified quatity of product|
|Stock25G_Saitama_Kawaguchi|TEXT|-|Number of goods available in Saitama_Kawaguchi|
|Stock25G_Hyogo_Amagasaki|TEXT|-|Number of goods available in Hyogo_Amagasaki|
|Stock25G_Stock_in_other_WH|TEXT|-| Number of goods in other Warehouses|

In [61]:
df_2.to_sql('available_stock', engine,  index=False)

In [62]:
# define a primary Key
with engine.connect() as con:
    con.execute('ALTER TABLE available_stock ADD PRIMARY KEY ("id");')

In [64]:
%%time
table_retrieved_2 = engine.execute("SELECT * FROM available_stock").fetchall()

CPU times: user 226 ms, sys: 2.43 ms, total: 229 ms
Wall time: 7.2 s


In [89]:
sql = "SELECT *\
       FROM compound_identification\
       INNER JOIN available_stock ON compound_identification.id=available_stock.id\
       WHERE rdkit_smiles>'[n,o,s]1cccc1';"

In [90]:
%%time
df_retrieved = pd.read_sql(sql, engine)

CPU times: user 25.4 ms, sys: 2.98 ms, total: 28.4 ms
Wall time: 1.83 s


In [93]:
df_sample = df_retrieved.sample()
df_sample.dropna(axis=1)

,name,cas,reaxys_registry_number,pubchem_id,mdl_number,pubchem_smiles,id,rdkit_smiles,id,Stock25G_price,Stock25G_Saitama_Kawaguchi,Stock25G_Hyogo_Amagasaki,Stock25G_Stock_in_other_WH
129,Angelicin,523-50-2,153970.0,172089135.0,mfcd00064930,C1=CC2=C(C=CO2)C3=C1C=CC(=O)O3,A2573,O=c1ccc2ccc3occc3c2o1,A2573,"47,200",Contact Company,8,Contact Company
